### Import packages correction

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import tensorflow as tf

In [3]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [4]:
from data.dataset import H5PY_RW
from data.data_tools import read_metadata, males_keys, females_keys

H5_dic = read_metadata()

chunk_size = 512*10

males = H5PY_RW('test_raw_16k.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size) # males voices dataset
females = H5PY_RW('test_raw_16k.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size=chunk_size) # females voices dataset

We are on PC


In [5]:
from data.dataset import Mixer

# Use validation set
mix_mf = Mixer([males, females], with_mask=False, with_inputs=True, shuffling=True).select_split(2)
mix_mm = Mixer([males, males], with_mask=False, with_inputs=True, shuffling=True).select_split(2)
mix_ff = Mixer([females, females], with_mask=False, with_inputs=True, shuffling=True).select_split(2)

mixers = [mix_mf, mix_ff, mix_ff]

In [11]:
from utils.bss_eval import bss_eval_sources
import numpy as np
## implement model loading directly, SavedModel since we don't have to modify the architecture here.

path = '/home/anthony/das/log/DPCL_finetuning/AdaptiveNet-royal-pond-5964-N=256--alpha=0.001--batch_size=1--beta=0.1--chunk_size=5120--maxpool=128--reg=0.0001--rho=0.01--smooth_size=20--type=DPCL_finetuning--window=1024-/SavedModel'
nb_iterations = 10

sdr = [0.0 for i in range(len(mixers))]
sir = [0.0 for i in range(len(mixers))]
sar = [0.0 for i in range(len(mixers))]

with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ['validating'], path)
#     show_graph(tf.get_default_graph().as_graph_def())
#     print tf.get_default_graph().get_operations()
    for j, mixer in enumerate(mixers):
        print 'z'
        for i in range(nb_iterations):
            non_mix_input, mix_input , ind = mixer.get_batch(1)
            
            input = tf.get_default_graph().get_operation_by_name('mix_input').values()[0]
            non_input = tf.get_default_graph().get_operation_by_name('non_mix_input').values()[0]
            back = tf.get_default_graph().get_operation_by_name('back/back_output').values()[0]
            is_training = tf.get_default_graph().get_operation_by_name('is_training').values()[0]
            test = tf.get_default_graph().get_operation_by_name('front/MaxPoolWithArgmax').values()[0]
            unmix = sess.run(back, feed_dict={input: mix_input, non_input: non_mix_input,is_training:False})
            
            unmix = np.reshape(unmix, (2, chunk_size))
            mix_input = np.reshape(mix_input, (chunk_size))
            mix_stack = np.array([mix_input, mix_input])
            
            no_separation = bss_eval_sources(np.squeeze(non_mix_input), mix_stack)
            
            separation = bss_eval_sources(np.squeeze(non_mix_input), unmix)
            
            sdr[j] += np.mean(separation[0] - no_separation[0])
            sir[j] += np.mean(separation[1] - no_separation[1])
            sar[j] += np.mean(separation[2] - no_separation[2])
        
        sdr[j] /= float(nb_iterations) 
        sir[j] /= float(nb_iterations) 
        sar[j] /= float(nb_iterations)
print sdr, sir, sar


        
        
        
    

INFO:tensorflow:Restoring parameters from /home/anthony/das/log/DPCL_finetuning/AdaptiveNet-royal-pond-5964-N=256--alpha=0.001--batch_size=1--beta=0.1--chunk_size=5120--maxpool=128--reg=0.0001--rho=0.01--smooth_size=20--type=DPCL_finetuning--window=1024-/SavedModel/variables/variables
z
z
z
[-1.7332590225796352, -1.1553780162181404, -3.1974157889507131] [0.73670325590756902, 1.6137457178537065, -0.12202363946564318] [-252.20321786861169, -260.09549921550968, -249.89236130090981]
